In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [2]:
#host settings
es = Elasticsearch('http://eis-warpcore-01:49200/')
docType = "doc"
indexName = "wikiindex"

In [3]:
es.count()

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'count': 6526329}

In [4]:
res = es.search(index=indexName, doc_type=docType)
print(res['hits'])

{'total': 6526329, 'hits': [{'_source': {'label': '[Impact of neoadjuvant hormonal therapy on the permanent \\u00B9\\u00B2\\u2075I-seed brachytherapy for localized high-risk prostate cancer].', 'knownAs': ''}, '_score': 1.0, '_id': 'oneoV2cBTRpO6fN9VGLE', '_type': 'doc', '_index': 'wikiindex'}, {'_source': {'label': 'Antioxidant activity and polyphenol content in edible wild fruits from Nepal.', 'knownAs': ''}, '_score': 1.0, '_id': 'lXeoV2cBTRpO6fN9VGKr', '_type': 'doc', '_index': 'wikiindex'}, {'_source': {'label': 'Cancer-preventing attributes of probiotics: an update.', 'knownAs': ''}, '_score': 1.0, '_id': 'iXeoV2cBTRpO6fN9VGKb', '_type': 'doc', '_index': 'wikiindex'}, {'_source': {'label': 'Template:Endflatlist', 'knownAs': ''}, '_score': 1.0, '_id': 'ZneoV2cBTRpO6fN9VGJR', '_type': 'doc', '_index': 'wikiindex'}, {'_source': {'label': 'Long Mountain (Jamaica)', 'knownAs': ''}, '_score': 1.0, '_id': 'TXeoV2cBTRpO6fN9VGIj', '_type': 'doc', '_index': 'wikiindex'}, {'_source': {'labe

In [ ]:
def entitySearch(query):
    results=[]
    ###################################################
    elasticResults=es.search(index=indexName, doc_type=docType, body={
              "query": {
                "prefix" : { "uri" : "http://dbpedia.org/resource/"+query.capitalize().replace(" ", "_") } 
              }
               ,"size":5
    }
           )
    for result in elasticResults['hits']['hits']:
        if result["_source"]["uri"].lower()=="http://dbpedia.org/resource/"+query.replace(" ", "_").lower():
            results.append([result["_source"]["label"],result["_source"]["uri"],result["_score"]*10,40])
        else:
            results.append([result["_source"]["label"],result["_source"]["uri"],result["_score"]*10,0])
        ###################################################
    elasticResults=es.search(index=indexName, doc_type=docType, body={
              "query": {
                "match" : { "uri" : "http://dbpedia.org/resource/"+query.capitalize().replace(" ", "_") } 
              }
               ,"size":5
    }
           )
    for result in elasticResults['hits']['hits']:
        if result["_source"]["uri"].lower()=="http://dbpedia.org/resource/"+query.replace(" ", "_").lower():
            results.append([result["_source"]["label"],result["_source"]["uri"],result["_score"]*20,40])
        else:
            results.append([result["_source"]["label"],result["_source"]["uri"],result["_score"]*20,0])
        ###################################################
    elasticResults=es.search(index=indexName, doc_type=docType, body={
              "query": {
                "match" : { "label" : query } 
              }
               ,"size":10
    }
           )
    for result in elasticResults['hits']['hits']:
        if result["_source"]["uri"].lower()=="http://dbpedia.org/resource/"+query.replace(" ", "_").lower():
            results.append([result["_source"]["label"],result["_source"]["uri"],result["_score"]*40,40])
        else:
            results.append([result["_source"]["label"],result["_source"]["uri"],result["_score"]*40,0])
    ###################################################
    elasticResults=es.search(index=indexName, doc_type=docType, body={
              "query": {
                "fuzzy" : { "label" : query  } 
              }
               ,"size":5
    }
           )
    for result in elasticResults['hits']['hits']:
        if result["_source"]["uri"].lower()=="http://dbpedia.org/resource/"+query.replace(" ", "_").lower():
            results.append([result["_source"]["label"],result["_source"]["uri"],result["_score"]*25,40])
        else:
            results.append([result["_source"]["label"],result["_source"]["uri"],result["_score"]*25,0])
    return results
    #for result in results['hits']['hits']:
        #print (result["_score"])
        #print (result["_source"])
        #print("-----------")

In [5]:
es.indices.refresh(index=indexName)

{'_shards': {'failed': 0, 'successful': 5, 'total': 10}}

In [9]:
res = es.search(index=indexName, body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print("%(label)s %(knownAs)s" % hit["_source"])

Got 6526329 Hits:
[Impact of neoadjuvant hormonal therapy on the permanent \u00B9\u00B2\u2075I-seed brachytherapy for localized high-risk prostate cancer]. 
Antioxidant activity and polyphenol content in edible wild fruits from Nepal. 
Cancer-preventing attributes of probiotics: an update. 
Template:Endflatlist 
Long Mountain (Jamaica) 
Hearing loss on social media: Who is winning hearts and minds? 
Dietary patterns and risk of oesophageal squamous cell carcinoma: a case-control study. 
[A case of rectal carcinoid with multiple liver, lymph nodes and bone metastases that responded to an octreotide therapy]. 
Pharmocokinetics and pharmacodynamics of single-dose triazolam: electroencephalography compared with the Digit-Symbol Substitution Test 
Guzmania candelabrum 


In [16]:
body = {
  "query": {
    "match": {
    "label": "United States of America"
             }
           }
,   "size": 100
}

res = es.search(index=indexName, body=body)
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print("%(label)s %(knownAs)s" % hit["_source"])

Got 2009675 Hits:
United States of America USA
United States of America the United States
United States of America US
United States of America U.S.
United States of America US
United States of America America
United States of America America
United States of America US
United States of America the States
United Steelworkers of America v. United States 
Category:Rocky Mountains (United States of America) 
Brimrose Corporation of America (United States) 
Australian Legation, United States of America 
United States\u2013Central America Treaty 
Category:Languages of the United States of America 
Music of the United States of America 
United States of America v. Jane Kember 
United States of America v. Sufyian Barhoumi 
United States of America: health system review 
Ellis v. United States of America (1969) 
1990 UNITED STATES OF AMERICA FARM BILL 
Microsoft Corporation v. United States of America 
Our surgical heritage. United States of America. 
United States of America v Kent Hovind 
Apo